In [5]:
import pandas as pd
import numpy as np
import feather
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error

### load true data

In [2]:
train = feather.read_dataframe('../data/interim/train.ftr')
test = feather.read_dataframe('../data/interim/test.ftr')
train.shape, test.shape

((903653, 55), (804684, 53))

In [3]:
# make target
train_index = np.sort(train['fullVisitorId'].unique())
test_index = np.sort(test['fullVisitorId'].unique())

train['target'] = train['totals.transactionRevenue'].fillna(0).astype('float').values
train_user_target = train[['fullVisitorId', 'target']].groupby('fullVisitorId').sum()
train_user_target = train_user_target.loc[train_index]
y_train = np.log1p(train_user_target['target'])

### load pred data

In [4]:
target_1 = "output_37"
target_2 = "output_16"

target_list = [target_1, target_2]
oof_preds_all = np.zeros([train_index.shape[0], len(target_list)])
sub_preds_all = np.zeros([test_index.shape[0], len(target_list)])
oof_preds_all.shape, sub_preds_all.shape

((714167, 2), (617242, 2))

In [7]:
for i, target in enumerate(target_list):
    pred_data = np.load(f"../data/interim/pred_user_{target}.npz")
    oof_preds_all[:, i] = pred_data["oof_preds"]
    sub_preds_all[:, i] = pred_data["sub_preds"]
    print(target, mean_squared_error(y_train, oof_preds_all[:, i])**0.5)

output_37 1.6024663112227862
output_16 1.5584814871280166


In [65]:
reg = ElasticNetCV(cv=5, random_state=0, fit_intercept=False, normalize=True,
                   l1_ratio=[.1, .2, .3, .4, .5, .7, .9, .95, .99, 1])
reg.fit(oof_preds_all, y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=False,
       l1_ratio=[0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.9, 0.95, 0.99, 1],
       max_iter=1000, n_alphas=100, n_jobs=1, normalize=True,
       positive=False, precompute='auto', random_state=0,
       selection='cyclic', tol=0.0001, verbose=0)

In [66]:
reg.intercept_, reg.alpha_, reg.coef_, reg.l1_ratio_

(0.0, 0.003143953758276617, array([0.27283017, 0.75160602]), 0.7)

In [59]:
mean_squared_error(y_train, reg.predict(oof_preds_all))**0.5

1.5524767961820864

### make submit

In [63]:
# save
sub_full_data = pd.DataFrame(index=test_index)
sub_full_data.index.name = "fullVisitorId"
sub_full_data['PredictedLogRevenue'] = reg.predict(sub_preds_all)
save_path = '../data/output/output_38_user.csv'
sub_full_data[['PredictedLogRevenue']].to_csv(save_path, index=True)

In [64]:
reg.predict(sub_preds_all).mean()

0.18773413712181194

In [61]:
sub_preds_all[:, 0].mean()

0.1833746487391711

In [62]:
sub_preds_all[:, 1].mean()

0.19254876373718252